In [176]:
%run -i ../header.py
import re

In [177]:
def norm(pname, outdirname):
    cfg = json.load(open(os.path.join("output", outdirname, "config"), "r"), object_pairs_hook=OrderedDict)
    prefix = outdirname.split(".")[0]
    points = pd.read_csv(os.path.join("../quarkU/output", prefix+"."+cfg["system"], "data", pname), index_col=0)
    return sp.sqrt(sp.integrate.trapz(points["u"]**2, points.index.values))

In [178]:
def plot(ax, pname, outdirname):
    cfg = json.load(open(os.path.join("output", outdirname, "config"), "r"), object_pairs_hook=OrderedDict)
    prefix = outdirname.split(".")[0]
    points = pd.read_csv(os.path.join("../quarkU/output", prefix+"."+cfg["system"], "data", pname), index_col=0)
    norm = sp.sqrt(sp.integrate.trapz(points["u"]**2, points.index.values))
    f = sp.interpolate.interp1d(points.index.values, points["u"])
    ax.plot(points.index.values*0.19732697, points["u"]**2/norm**2/0.19732697, ".")

In [179]:
def overlap(pname1, pname2, outdirname, func = lambda r: r):
    cfg = json.load(open(os.path.join("output", outdirname, "config"), "r"), object_pairs_hook=OrderedDict)
    prefix = outdirname.split(".")[0]
    points1 = pd.read_csv(os.path.join("../quarkU/output", prefix+"."+cfg["system"], "data", pname1), index_col=0)
    points2 = pd.read_csv(os.path.join("../quarkU/output", prefix+"."+cfg["system"], "data", pname2), index_col=0)
    f1 = sp.interpolate.interp1d(points1.index.values, points1["u"])
    f2 = sp.interpolate.interp1d(points2.index.values, points2["u"])
    if (points1.index.values[-1] > points2.index.values[-1]):
        xes = points2.index.values
    else:
        xes = points1.index.values
    return sp.integrate.trapz(f1(xes)*f2(xes)*func(xes), xes)/sp.sqrt(sp.integrate.trapz(points1["u"]**2, points1.index.values)*sp.integrate.trapz(points2["u"]**2, points2.index.values))

In [180]:
def readCSV(outdirname):
    data = pd.read_csv(open(os.path.join("output", outdirname, "data")), index_col=[0,1])
    cfg = json.load(open(os.path.join("output", outdirname, "eigen_config"), "r"), object_pairs_hook=OrderedDict)
    def k(row):
        Mf = 2*cfg[row.name[1]]["eq"]["env"]["mC"]+cfg[row.name[1]]["eq"]["E"]
        return -Mf+sp.sqrt(Mf**2 + 2*Mf*(cfg[row.name[0]]["eq"]["E"] - cfg[row.name[1]]["eq"]["E"]))
    lap1 = data.apply(lambda r: overlap(*r.name, outdirname), axis=1).rename("melLW")
    lapful = data.apply(lambda r: overlap(*r.name, outdirname, lambda x: x*(sp.special.spherical_jn(0, k(r)*x/2) - sp.sqrt(5/2)*sp.special.spherical_jn(2, k(r)*x/2))), axis=1).rename("melE1")
    gamma = data.apply(k, axis=1).rename("$E_\gamma$")
    data = pd.concat([data, lap1, lapful, gamma], axis=1)
    check = ((data["widthE1"]/data["melE1"]**2-data["widthELW"]/data["melLW"]**2)*2/(data["widthE1"]/data["melE1"]**2+data["widthELW"]/data["melLW"]**2)).rename("check")
    data = pd.concat([data, check], axis=1)
    return data

In [181]:
def assembleData(datapath, paperpath):
    chin = pd.read_csv(paperpath, index_col=[0,1]).rename(columns={"Gtot": "widthPAPER"})
    report = readCSV(datapath)
    report = pd.concat([report, chin], axis=1)
    
    return report

In [182]:
def relativeQuant(report):
    return pd.DataFrame(index=report.index)\
        .join(\
            (report["widthPAPER"]/report["widthE1"])\
                .rename("PAPER/E1"))\
        .join(\
              (report["widthPAPER"]/report["width"])\
                  .rename("PAPER/width"))\
        .join(\
              (report["widthPAPER"]/report["width_mod"])\
                  .rename("PAPER/width_mod"))\
        .join(\
              (report["widthPAPER"]/report["widthE1_mod"])\
                  .rename("PAPER/widthE1_mod"))\
        .join(\
              (report["widthPAPER"]/report["widthELW_mod"])\
                  .rename("PAPER/widthELW_mod")\
             )

In [183]:
def experimentalReport(report, particledata, transdata):
    trans = pd.read_csv(transdata, index_col=[0, 1]).add_suffix("_pdg")
    pdata = pd.read_csv(particledata, index_col=0)
    width = trans.apply(lambda r: r["frac_width_pdg"]*pdata.loc[r.name[0], "PDG_width"],axis=1).rename("width_pdg")
    eMix = report["$E_\gamma$"].copy()
    collection = []
    collection.append(report[["width", "widthE1", "widthELW", "$E_\gamma$", "widthPAPER"]])
    try:
        notnanE = sp.logical_not(sp.isnan(trans["egamma_pdg"]))
        notnanE = [i for i in notnanE[notnanE].index.values for j in report.index.values if i == j]
        eMix[notnanE] = trans["egamma_pdg"][notnanE].copy()
        collection.append(trans[["egamma_pdg"]])
    except KeyError:
        pass
    eMix.rename("E", inplace=True)
    
    MMix = pdata["PDG"].copy()
    nanM = sp.isnan(MMix)
    nanM = nanM[nanM].index.values
    MMix[nanM] = pdata["SP"][nanM].copy()
    MMix /= 1000
    
    expfactor = eMix.index.map(\
              lambda row: eMix[row]*MMix[row[0]]/(eMix[row] + MMix[row[0]])*(MMix[row[0]] - MMix[row[1]])**2\
        )
    expfactor = pd.DataFrame(expfactor.values, index=eMix.index)
    
    width_mod = (expfactor[0]*report["R_width"].copy()).rename("width_mod")
    widthE1_mod = (expfactor[0]*report["R_widthE1"].copy()).rename("widthE1_mod")
    widthELW_mod = (expfactor[0]*report["R_widthELW"].copy()).rename("widthELW_mod")
    collection += [width_mod, widthE1_mod, widthELW_mod]
    
    return pd.concat(collection, join="inner", axis=1).join(width)

In [184]:
def doReport(datapath, particledata, paperpath, transdata):
    data = assembleData(datapath, paperpath)
    exper = experimentalReport(data, particledata, transdata)
    rel = (relativeQuant(exper)-1.)*100
    rel.rename(columns={i: i+"(%)" for i in rel.columns}, inplace=True)
    
    labels = ["width", "widthE1", "widthELW", "$E_\gamma$", "widthPAPER"]
    for labe in ["Ge1", "Gm1"]:
        if labe in data.columns:
            labels.append(labe)
    data[labels] *= 10**6
    data.rename(columns=dict(zip(labels, ["%s(KeV)"%i for i in labels])), inplace=True)
    
    labels = ["melLW", "melE1"]
    data[labels] *= 10**(-6)
    data.rename(columns=dict(zip(labels, ["%s($KeV^{-1}$)"%i for i in labels])), inplace=True)
    
    labels = ["R_width", "R_widthE1", "R_widthELW"]
    data[labels] *= 10**(-12)
    data.rename(columns=dict(zip(labels, ["%s($KeV^{-2}$)"%i for i in labels])), inplace=True)
    
    labels = ["width", "widthE1", "widthELW", "width_mod", "widthE1_mod", "widthELW_mod", "$E_\gamma$", "widthPAPER", "width_pdg"]
    for labe in ["egamma_pdg"]:
        if labe in exper.columns:
            labels.append(labe)
    exper[labels] *= 10**6
    exper.rename(columns=dict(zip(labels, ["%s(KeV)"%i for i in labels])), inplace=True)
    
    display(data)
    display(exper)
    display(rel)

# Export

In [185]:
def cfgnameSplit(cfgname):
    cfgcode = cfgname.split(".")
    cfgtag = "" if len(cfgcode) == 1 else ".".join(cfgcode[:-1])
    cfgcode = cfgcode[-1]
    return cfgcode, cfgtag

def getPDGTrans(name):
    pdg_trans = pd.read_csv("../data/{}-trans-PDG.csv".format(name), header=0, index_col=[0,1])["frac_width"]\
                  .rename("PDG")
    pdg = pd.read_csv("../data/{}-spectrum.csv".format(name), header=0, index_col=0)["PDG_width"]
    for instate in pdg_trans.index.levels[0]:
        pdg_trans.loc[(instate, slice(None))] = pdg_trans[(instate, slice(None))].values*pdg[instate]*10**6
    pdg_trans.index = pd.Index([tuple("${}$".format(code2name(e)) for e in pair) for pair in pdg_trans.index.values], name=["In", "Out"])
    return pdg_trans

def getDengTrans(cfgcode, cfgtag=""):
    suffix = "" if cfgtag == "" else "-"+cfgtag
    deng = pd.read_csv("../data/{}.deng2017{}.csv".format(cfgcode, suffix), header=0, index_col=[0,1])
    if sp.any(pd.isna(deng["Gtot"])):
        deng.loc[pd.isna(deng["Gtot"]), "Gtot"] = deng.loc[pd.isna(deng["Gtot"]).index.values, "Ge1"]
    if sp.any(pd.isna(deng["Gtot"])):
        deng.loc[pd.isna(deng["Gtot"]), "Gtot"] = deng.loc[pd.isna(deng["Gtot"]).index.values, "Gm1"]
    deng = deng["Gtot"].rename("Deng")
    deng *= 10**6
    deng.index = pd.Index([tuple("${}$".format(code2name(e)) for e in pair) for pair in deng.index.values], name=["In", "Out"])
    return deng

def getRs(prefix, cfgname, suffix="", particle=None):
    data = pd.read_csv("output/{}.{}/widths{}".format(prefix, cfgname, suffix), header=0, index_col = [0, 1])
    data *= 10**6
    data["width0"] /= data["width"]
    data["width2"] /= data["width"]
    data.columns = ["$\Gamma$", "$r^{(0)}$", "$r^{(2)}$"]
    data.index.names = ["In", "Out"]
    data.index = pd.Index([tuple("${}$".format(code2name(e)) for e in pair) for pair in data.index.values], name=["In", "Out"])
    return data.query("Out == r'${0}$' | In == r'${0}$'".format(code2name(particle))) if particle is not None\
            else data
def reportRs(pname, prefix, cfgnames, suffixes=["", "E1", "ELW"]):
    stack = []
    deng_stack = []
    for cfgname in cfgnames:
        cfgcode, cfgtag = cfgnameSplit(cfgname)
        stack.append(pd.concat([getRs(prefix, cfgname, suffix, particle=pname)\
                                    .add_prefix("" if suffix == "" else suffix+"_")
                                for suffix in suffixes], axis=1))
        deng_stack.append(getDengTrans(cfgcode, cfgtag))
    res = pd.concat(stack)
    res.insert(0, "Deng", pd.concat(deng_stack))
    res.insert(0, "PDG", getPDGTrans(cfgcode.split("-")[0]))
    return res

## Widths

### $\psi(1S)$ and $\Upsilon(1S)$

In [186]:
res = reportRs("psi_1S", "scr", ("c-scr",))
display(res)

,,PDG,Deng,$\Gamma$,$r^{(0)}$,$r^{(2)}$,E1_$\Gamma$,E1_$r^{(0)}$,E1_$r^{(2)}$,ELW_$\Gamma$,ELW_$r^{(0)}$,ELW_$r^{(2)}$
In,Out,,,,,,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,2.44,2.78,0,0,2.78,0,0,2.8,0,0
$\eta_{c}(2S)$,$\psi(1S)$,nan,2.29,2.91,1,0,2.91,1,0,9.19,1,0
$\chi_{c2}(1P)$,$\psi(1S)$,371,292,311,0.0552,0.696,361,0.1,0.6,481,0.1,0.6
$\chi_{c1}(1P)$,$\psi(1S)$,285,319,376,0.434,0,326,0.5,0,411,0.5,0
$\chi_{c0}(1P)$,$\psi(1S)$,133,179,204,1,0,167,1,0,187,1,0


In [187]:
res = reportRs("psi_1S", "lin", ("c-lin",))
display(res)

,,PDG,Deng,$\Gamma$,$r^{(0)}$,$r^{(2)}$,E1_$\Gamma$,E1_$r^{(0)}$,E1_$r^{(2)}$,ELW_$\Gamma$,ELW_$r^{(0)}$,ELW_$r^{(2)}$
In,Out,,,,,,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,2.39,2.57,0,0,2.57,0,0,2.58,0,0
$\eta_{c}(2S)$,$\psi(1S)$,nan,2.64,3.03,1,0,3.03,1,0,8.66,1,0
$\chi_{c2}(1P)$,$\psi(1S)$,371,284,303,0.0576,0.691,349,0.1,0.6,458,0.1,0.6
$\chi_{c1}(1P)$,$\psi(1S)$,285,306,350,0.438,0,306,0.5,0,380,0.5,0
$\chi_{c0}(1P)$,$\psi(1S)$,133,172,195,1,0,161,1,0,180,1,0


In [188]:
res = reportRs("yps_1S", "scr", ("1P1D2D.b-scr", "nSnP.b-scr"))
display(res)

,,PDG,Deng,$\Gamma$,$r^{(0)}$,$r^{(2)}$,E1_$\Gamma$,E1_$r^{(0)}$,E1_$r^{(2)}$,ELW_$\Gamma$,ELW_$r^{(0)}$,ELW_$r^{(2)}$
In,Out,,,,,,,,,,,
$\chi_{b2}(1P)$,$\Upsilon(1S)$,nan,31.8,32.8,0.0858,0.629,34.4,0.1,0.6,37.8,0.1,0.6
$\chi_{b1}(1P)$,$\Upsilon(1S)$,nan,31.9,33.6,0.478,0,32.1,0.5,0,34.9,0.5,0
$\chi_{b0}(1P)$,$\Upsilon(1S)$,nan,27.5,28.3,1,0,26.1,1,0,27.9,1,0
$\chi_{b2}(2P)$,$\Upsilon(1S)$,nan,12.5,16.8,0.0786,0.644,18.1,0.1,0.6,13.8,0.1,0.6
$\chi_{b1}(2P)$,$\Upsilon(1S)$,nan,10.8,15,0.467,0,14,0.5,0,9.79,0.5,0
$\chi_{b0}(2P)$,$\Upsilon(1S)$,nan,5.54,7.97,1,0,7.07,1,0,3.79,1,0
$\chi_{b2}(3P)$,$\Upsilon(1S)$,nan,8.17,11.7,0.0739,0.654,12.7,0.1,0.6,8.1,0.1,0.6
$\chi_{b1}(3P)$,$\Upsilon(1S)$,nan,6.41,9.62,0.46,0,8.84,0.5,0,4.87,0.5,0
$\chi_{b0}(3P)$,$\Upsilon(1S)$,nan,1.87,3.71,1,0,3.24,1,0,1.01,1,0


### $\psi(2S)$ and $\Upsilon(2S)$

In [189]:
res = reportRs("psi_2S", "scr", ("c-scr",))
display(res)

PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In         Out                                                           
$\psi(2S)$ $\eta_{c}(2S)$  0.207  0.19     0.148          0          0   
           $\eta_{c}(1S)$   1.01   7.8      10.3          0          0   
           $\chi_{c2}(1P)$    27    46      42.8      0.313          0   
           $\chi_{c1}(1P)$  28.3    45      34.9      0.472          0   
           $\chi_{c0}(1P)$  29.6    22      20.2          0          0   

                            E1_$\Gamma$  E1_$r^{(0)}$  E1_$r^{(2)}$  \
In         Out                                                        
$\psi(2S)$ $\eta_{c}(2S)$         0.148             0             0   
           $\eta_{c}(1S)$          10.3             0             0   
           $\chi_{c2}(1P)$           41           0.3             0   
           $\chi_{c1}(1P)$         36.8           0.5             0   
           $\chi_{c0}(1P)$         24.5             0             0   

                            ELW_$\Gamma$  ELW_$r^{(0)}$  ELW_$r^{(2)}$  
In         Out                                                          
$\psi(2S)$ $\eta_{c}(2S)$          0.149              0              0  
           $\eta_{c}(1S)$           4.93              0              0  
           $\chi_{c2}(1P)$          43.2            0.3              0  
           $\chi_{c1}(1P)$          39.9            0.5              0  
           $\chi_{c0}(1P)$          30.3              0              0

In [190]:
res = reportRs("psi_2S", "lin", ("c-lin",))
display(res)

PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In         Out                                                           
$\psi(2S)$ $\eta_{c}(2S)$  0.207  0.19     0.158          0          0   
           $\eta_{c}(1S)$   1.01  8.08      9.63          0          0   
           $\chi_{c2}(1P)$    27    38      40.9      0.312          0   
           $\chi_{c1}(1P)$  28.3    42      35.9      0.472          0   
           $\chi_{c0}(1P)$  29.6    22        21          0          0   

                            E1_$\Gamma$  E1_$r^{(0)}$  E1_$r^{(2)}$  \
In         Out                                                        
$\psi(2S)$ $\eta_{c}(2S)$         0.158             0             0   
           $\eta_{c}(1S)$          9.63             0             0   
           $\chi_{c2}(1P)$         39.3           0.3             0   
           $\chi_{c1}(1P)$         37.8           0.5             0   
           $\chi_{c0}(1P)$         25.3             0             0   

                            ELW_$\Gamma$  ELW_$r^{(0)}$  ELW_$r^{(2)}$  
In         Out                                                          
$\psi(2S)$ $\eta_{c}(2S)$          0.158              0              0  
           $\eta_{c}(1S)$            4.8              0              0  
           $\chi_{c2}(1P)$          41.2            0.3              0  
           $\chi_{c1}(1P)$          40.9            0.5              0  
           $\chi_{c0}(1P)$          30.7              0              0

In [191]:
res = reportRs("yps_2S", "scr", ("1P1D2D.b-scr", "nSnP.b-scr"))
display(res)

PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In              Out                                                          
$\Upsilon(2S)$  $\chi_{b2}(1P)$ 2.29  2.62      1.59      0.303          0   
                $\chi_{b1}(1P)$ 2.21  2.17       1.4      0.494          0   
                $\chi_{b0}(1P)$ 1.22  1.09     0.861          0          0   
$\chi_{b2}(2P)$ $\Upsilon(2S)$   nan  15.3      15.4     0.0921      0.616   
$\chi_{b1}(2P)$ $\Upsilon(2S)$   nan  15.3      15.4      0.488          0   
$\chi_{b0}(2P)$ $\Upsilon(2S)$   nan  14.4        13          1          0   
$\chi_{b2}(3P)$ $\Upsilon(2S)$   nan  6.72      9.23     0.0865      0.628   
$\chi_{b1}(3P)$ $\Upsilon(2S)$   nan  5.63      8.18      0.479          0   
$\chi_{b0}(3P)$ $\Upsilon(2S)$   nan  2.55      4.54          1          0   

                                 E1_$\Gamma$  E1_$r^{(0)}$  E1_$r^{(2)}$  \
In              Out                                                        
$\Upsilon(2S)$  $\chi_{b2}(1P)$         1.58           0.3             0   
                $\chi_{b1}(1P)$         1.41           0.5             0   
                $\chi_{b0}(1P)$        0.889             0             0   
$\chi_{b2}(2P)$ $\Upsilon(2S)$          15.8           0.1           0.6   
$\chi_{b1}(2P)$ $\Upsilon(2S)$          15.1           0.5             0   
$\chi_{b0}(2P)$ $\Upsilon(2S)$          12.5             1             0   
$\chi_{b2}(3P)$ $\Upsilon(2S)$          9.65           0.1           0.6   
$\chi_{b1}(3P)$ $\Upsilon(2S)$          7.84           0.5             0   
$\chi_{b0}(3P)$ $\Upsilon(2S)$          4.23             1             0   

                                 ELW_$\Gamma$  ELW_$r^{(0)}$  ELW_$r^{(2)}$  
In              Out                                                          
$\Upsilon(2S)$  $\chi_{b2}(1P)$          1.59            0.3              0  
                $\chi_{b1}(1P)$          1.43            0.5              0  
                $\chi_{b0}(1P)$         0.909              0              0  
$\chi_{b2}(2P)$ $\Upsilon(2S)$           17.7            0.1            0.6  
$\chi_{b1}(2P)$ $\Upsilon(2S)$           16.5            0.5              0  
$\chi_{b0}(2P)$ $\Upsilon(2S)$           13.3              1              0  
$\chi_{b2}(3P)$ $\Upsilon(2S)$           6.26            0.1            0.6  
$\chi_{b1}(3P)$ $\Upsilon(2S)$           4.52            0.5              0  
$\chi_{b0}(3P)$ $\Upsilon(2S)$           1.69              1              0

### $\psi(3S)$ and $\Upsilon(3S)$

In [192]:
res = reportRs("psi_3S", "scr", ("c-scr",))
display(res)

PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In         Out                                                          
$\psi(3S)$ $\eta_{c}(3S)$   nan 0.088    0.0344          0          0   
           $\eta_{c}(2S)$   nan  1.78      2.01          0          0   
           $\eta_{c}(1S)$   nan  6.76      9.62          0          0   
           $\chi_{c2}(2P)$  nan    82      46.2       0.31          0   
           $\chi_{c1}(2P)$  nan    67      39.4      0.479          0   
           $\chi_{c0}(2P)$  nan    27      20.9          0          0   
           $\chi_{c2}(1P)$  nan   2.5      35.5      0.311          0   
           $\chi_{c1}(1P)$  nan   6.7      32.6       0.46          0   
           $\chi_{c0}(1P)$  nan   6.7      17.6          0          0   

                            E1_$\Gamma$  E1_$r^{(0)}$  E1_$r^{(2)}$  \
In         Out                                                        
$\psi(3S)$ $\eta_{c}(3S)$        0.0344             0             0   
           $\eta_{c}(2S)$          2.01             0             0   
           $\eta_{c}(1S)$          9.62             0             0   
           $\chi_{c2}(2P)$         44.7           0.3             0   
           $\chi_{c1}(2P)$           41           0.5             0   
           $\chi_{c0}(2P)$           24             0             0   
           $\chi_{c2}(1P)$         34.1           0.3             0   
           $\chi_{c1}(1P)$         35.2           0.5             0   
           $\chi_{c0}(1P)$         23.9             0             0   

                            ELW_$\Gamma$  ELW_$r^{(0)}$  ELW_$r^{(2)}$  
In         Out                                                          
$\psi(3S)$ $\eta_{c}(3S)$         0.0345              0              0  
           $\eta_{c}(2S)$          0.714              0              0  
           $\eta_{c}(1S)$            3.8              0              0  
           $\chi_{c2}(2P)$          47.9            0.3              0  
           $\chi_{c1}(2P)$          45.7            0.5              0  
           $\chi_{c0}(2P)$          31.4              0              0  
           $\chi_{c2}(1P)$          1.92            0.3              0  
           $\chi_{c1}(1P)$         0.675            0.5              0  
           $\chi_{c0}(1P)$          7.73              0              0

In [193]:
res = reportRs("psi_3S", "lin", ("c-lin",))
display(res)

PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In         Out                                                          
$\psi(3S)$ $\eta_{c}(3S)$   nan 0.051    0.0527          0          0   
           $\eta_{c}(2S)$   nan  1.65       2.8          0          0   
           $\eta_{c}(1S)$   nan  6.66      10.4          0          0   
           $\chi_{c2}(2P)$  nan    67      64.9      0.311          0   
           $\chi_{c1}(2P)$  nan    55      54.8      0.476          0   
           $\chi_{c0}(2P)$  nan    19      26.8          0          0   
           $\chi_{c2}(1P)$  nan  0.25      41.7      0.307          0   
           $\chi_{c1}(1P)$  nan     4      42.2      0.463          0   
           $\chi_{c0}(1P)$  nan   5.9      22.9          0          0   

                            E1_$\Gamma$  E1_$r^{(0)}$  E1_$r^{(2)}$  \
In         Out                                                        
$\psi(3S)$ $\eta_{c}(3S)$        0.0527             0             0   
           $\eta_{c}(2S)$           2.8             0             0   
           $\eta_{c}(1S)$          10.4             0             0   
           $\chi_{c2}(2P)$         62.5           0.3             0   
           $\chi_{c1}(2P)$         57.5           0.5             0   
           $\chi_{c0}(2P)$         31.2             0             0   
           $\chi_{c2}(1P)$         40.8           0.3             0   
           $\chi_{c1}(1P)$         45.3           0.5             0   
           $\chi_{c0}(1P)$         30.5             0             0   

                            ELW_$\Gamma$  ELW_$r^{(0)}$  ELW_$r^{(2)}$  
In         Out                                                          
$\psi(3S)$ $\eta_{c}(3S)$         0.0528              0              0  
           $\eta_{c}(2S)$          0.894              0              0  
           $\eta_{c}(1S)$           4.22              0              0  
           $\chi_{c2}(2P)$          67.7            0.3              0  
           $\chi_{c1}(2P)$          65.4            0.5              0  
           $\chi_{c0}(2P)$          41.9              0              0  
           $\chi_{c2}(1P)$           7.1            0.3              0  
           $\chi_{c1}(1P)$         0.105            0.5              0  
           $\chi_{c0}(1P)$           7.3              0              0

In [194]:
res = reportRs("yps_3S", "scr", ("1P1D2D.b-scr", "nSnP.b-scr"))
display(res)

PDG   Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In              Out                                                             
$\Upsilon(3S)$  $\chi_{b2}(2P)$   2.66   3.16      2.44      0.302          0   
                $\chi_{b1}(2P)$   2.56   2.61      2.15      0.495          0   
                $\chi_{b0}(2P)$    1.2   1.21      1.23          0          0   
                $\chi_{b2}(1P)$  0.201   0.14  0.000345      0.467          0   
                $\chi_{b1}(1P)$ 0.0183 0.0005    0.0841      0.501          0   
                $\chi_{b0}(1P)$ 0.0549  0.097     0.209          0          0   
$\chi_{b2}(3P)$ $\Upsilon(3S)$     nan   10.8      12.1      0.094      0.612   
$\chi_{b1}(3P)$ $\Upsilon(3S)$     nan   10.3      11.9      0.491          0   
$\chi_{b0}(3P)$ $\Upsilon(3S)$     nan   7.95      9.64          1          0   

                                 E1_$\Gamma$  E1_$r^{(0)}$  E1_$r^{(2)}$  \
In              Out                                                        
$\Upsilon(3S)$  $\chi_{b2}(2P)$         2.42           0.3             0   
                $\chi_{b1}(2P)$         2.18           0.5             0   
                $\chi_{b0}(2P)$         1.26             0             0   
                $\chi_{b2}(1P)$     4.06e-05           0.3             0   
                $\chi_{b1}(1P)$        0.084           0.5             0   
                $\chi_{b0}(1P)$        0.222             0             0   
$\chi_{b2}(3P)$ $\Upsilon(3S)$          12.3           0.1           0.6   
$\chi_{b1}(3P)$ $\Upsilon(3S)$          11.7           0.5             0   
$\chi_{b0}(3P)$ $\Upsilon(3S)$          9.34             1             0   

                                 ELW_$\Gamma$  ELW_$r^{(0)}$  ELW_$r^{(2)}$  
In              Out                                                          
$\Upsilon(3S)$  $\chi_{b2}(2P)$          2.46            0.3              0  
                $\chi_{b1}(2P)$          2.22            0.5              0  
                $\chi_{b0}(2P)$          1.31              0              0  
                $\chi_{b2}(1P)$         0.394            0.3              0  
                $\chi_{b1}(1P)$        0.0357            0.5              0  
                $\chi_{b0}(1P)$        0.0394              0              0  
$\chi_{b2}(3P)$ $\Upsilon(3S)$             14            0.1            0.6  
$\chi_{b1}(3P)$ $\Upsilon(3S)$             13            0.5              0  
$\chi_{b0}(3P)$ $\Upsilon(3S)$             10              1              0

## Sumrules

In [260]:
def getMasses(prefix, cfgname):
    with open("../quarkEigen/output/{}.{}/config".format(prefix, cfgname), "r") as f:
        mass_cfg = json.load(f, object_pairs_hook=OrderedDict)
    with open("../quarkEigen/output/{}.{}/exclude".format(prefix, cfgname), "r") as f:
        mass_excl = f.read().split("\n")
    mass_data = OrderedDict()
    for particle, pdata in mass_cfg.items():
        if particle not in mass_excl:
            continue
        energ_data = pd.read_csv("../quarkEigen/output/{}.{}/data/{}/minE.dat".format(prefix, cfgname, particle))\
                     .iloc[-1, 1]
        energ = pdata["eq"]["env"]["mC"]*2 + energ_data
        mass_data.update({particle: energ})
    cfgcode, cfgtag = cfgnameSplit(cfgname)
    mass_data = pd.DataFrame.from_dict(mass_data, orient="index")
    mass_data.columns = ["E"]
    pdg = pd.read_csv("../data/{}-spectrum.csv".format(cfgcode.split("-")[0]), header=0, index_col=0)["PDG"]/10**3
    pdg = pdg.loc[pdg.index.intersection(mass_data.index).values]
    mass_data.loc[~pd.isna(pdg), "E"] = pdg.loc[~pd.isna(pdg)]
    mass_data.index = pd.Index(["${}$".format(code2name(k)) for k in mass_data.index.values], name="States")
    return mass_data

In [261]:
def getPprops(prefix, cfgname):
    with open("../quarkEigen/output/{}.{}/config".format(prefix, cfgname), "r") as f:
        pprops = json.load(f, object_pairs_hook=OrderedDict)
    props = pd.DataFrame.from_dict({particle: values["eq"] for particle, values in pprops.items()}
                        ,orient="index")
    props.drop(columns=["env", "E"], inplace=True)
    props.index = pd.Index([r"${}$".format(code2name(particle)) for particle in pprops.keys()], name="States")
    return props

In [304]:
def getSumrule(pname, prefix, cfgnames, suffix=""):
    cfgcode, cfgtag = cfgnameSplit(cfgnames[0])
    msuffix = "" if suffix == "" else suffix+"_"
    pretty_pname = "${}$".format(code2name(pname))
    
    widths = reportRs(pname, prefix, cfgnames, [suffix])
    masses = getMasses(prefix, cfgcode)
    pprops = getPprops(prefix, cfgcode)
    
    sumrule = []

    abovethr = [p[0] for p in widths.query("Out == r'{}'".format(pretty_pname)).index.values]
    sumrule.append(
        8*sp.pi**2*masses.loc[abovethr, "E"]**3\
        /(masses.loc[abovethr, "E"]**2 - masses.loc[pretty_pname, "E"]**2)**3\
        *pprops.loc[abovethr, "xJ"]
    )
    sumrule[0].index = pd.Index([(v, pretty_pname) for v in sumrule[0].index.values])
    
    belowthr = [p[1] for p in widths.query("In == r'{}'".format(pretty_pname)).index.values]
    sumrule.append(
        -8*sp.pi**2*masses.loc[pretty_pname, "E"]**3\
        /(masses.loc[pretty_pname, "E"]**2 - masses.loc[belowthr, "E"]**2)**3\
        *pprops.loc[pretty_pname, "xJ"]
    )
    sumrule[1].index = pd.Index([(pretty_pname, v) for v in sumrule[1].index.values])
    sr = pd.concat(sumrule)*(0.19732697)**2*10**4*10**(-6)
    sr = sr.reindex(widths.index.values)
    
    widths["$\sigma_{PDG}$"] = widths["PDG"]
    if sp.any(pd.isna(widths["PDG"])):
        widths.loc[pd.isna(widths["PDG"]), "$\sigma_{PDG}$"] = widths.loc[pd.isna(widths["PDG"]).index.values, msuffix+"$\Gamma$"]
    widths.loc[:, "$\sigma_{PDG}$"] *= sr*(widths.loc[:, msuffix+"$r^{(2)}$"] - widths.loc[:, msuffix+"$r^{(0)}$"])
    
    widths["$\sigma_{Deng}$"] = widths["Deng"]
    if sp.any(pd.isna(widths["Deng"])):
        widths.loc[pd.isna(widths["Deng"]), "$\sigma_{Deng}$"] = widths.loc[pd.isna(widths["Deng"]).index.values, msuffix+"$\Gamma$"]
    widths.loc[:, "$\sigma_{Deng}$"] *= sr*(widths.loc[:, msuffix+"$r^{(2)}$"] - widths.loc[:, msuffix+"$r^{(0)}$"])
    
    widths["$\sigma_{\Gamma}$"] = widths[msuffix+"$\Gamma$"]*sr*(widths.loc[:, msuffix+"$r^{(2)}$"] - widths.loc[:, msuffix+"$r^{(0)}$"])
    
    widths.drop(columns=[msuffix+'$r^{(2)}$', msuffix+'$r^{(0)}$'], inplace=True)
    widths = widths[[widths.columns[i] for i in [0, 3, 1, 4, 2, 5]]]
    return widths, pd.DataFrame(widths[["$\sigma_{PDG}$", "$\sigma_{Deng}$", "$\sigma_{\Gamma}$"]].sum().rename("$\sigma$"))

## Bottomonium sumrules

### $\Upsilon(1S)$

In [305]:
report, sr = getSumrule("yps_1S", "scr", ("1P1D2D.b-scr", "nSnP.b-scr"), "ELW")
display(sr)
display(report)

,$\sigma$
$\sigma_{PDG}$,-1.94
$\sigma_{Deng}$,-2.26
$\sigma_{\Gamma}$,-1.94


,,PDG,$\sigma_{PDG}$,Deng,$\sigma_{Deng}$,ELW_$\Gamma$,$\sigma_{\Gamma}$
In,Out,,,,,,
$\chi_{b2}(1P)$,$\Upsilon(1S)$,nan,4.22,31.8,3.55,37.8,4.22
$\chi_{b1}(1P)$,$\Upsilon(1S)$,nan,-4.41,31.9,-4.03,34.9,-4.41
$\chi_{b0}(1P)$,$\Upsilon(1S)$,nan,-1.79,27.5,-1.77,27.9,-1.79
$\chi_{b2}(2P)$,$\Upsilon(1S)$,nan,0.0567,12.5,0.0512,13.8,0.0567
$\chi_{b1}(2P)$,$\Upsilon(1S)$,nan,-0.0422,10.8,-0.0465,9.79,-0.0422
$\chi_{b0}(2P)$,$\Upsilon(1S)$,nan,-0.0354,5.54,-0.0517,3.79,-0.0354
$\chi_{b2}(3P)$,$\Upsilon(1S)$,nan,0.104,8.17,0.105,8.1,0.104
$\chi_{b1}(3P)$,$\Upsilon(1S)$,nan,-0.0281,6.41,-0.037,4.87,-0.0281
$\chi_{b0}(3P)$,$\Upsilon(1S)$,nan,-0.0124,1.87,-0.0229,1.01,-0.0124


### $\Upsilon(2S)$

In [306]:
report, sr = getSumrule("yps_2S", "scr", ("1P1D2D.b-scr", "nSnP.b-scr"), "ELW")
display(sr)
display(report)

,$\sigma$
$\sigma_{PDG}$,-1.78
$\sigma_{Deng}$,-2.22
$\sigma_{\Gamma}$,-3.07


PDG  $\sigma_{PDG}$  Deng  $\sigma_{Deng}$  \
In              Out                                                           
$\Upsilon(2S)$  $\chi_{b2}(1P)$ 2.29            1.96  2.62             2.25   
                $\chi_{b1}(1P)$ 2.21            1.97  2.17             1.94   
                $\chi_{b0}(1P)$ 1.22              -0  1.09               -0   
$\chi_{b2}(2P)$ $\Upsilon(2S)$   nan            2.36  15.3             2.05   
$\chi_{b1}(2P)$ $\Upsilon(2S)$   nan           -2.63  15.3            -2.44   
$\chi_{b0}(2P)$ $\Upsilon(2S)$   nan            -5.7  14.4            -6.16   
$\chi_{b2}(3P)$ $\Upsilon(2S)$   nan           0.703  6.72            0.754   
$\chi_{b1}(3P)$ $\Upsilon(2S)$   nan          -0.239  5.63           -0.298   
$\chi_{b0}(3P)$ $\Upsilon(2S)$   nan          -0.203  2.55           -0.306   

                                 ELW_$\Gamma$  $\sigma_{\Gamma}$  
In              Out                                               
$\Upsilon(2S)$  $\chi_{b2}(1P)$          1.59               1.37  
                $\chi_{b1}(1P)$          1.43               1.28  
                $\chi_{b0}(1P)$         0.909                 -0  
$\chi_{b2}(2P)$ $\Upsilon(2S)$           17.7               2.36  
$\chi_{b1}(2P)$ $\Upsilon(2S)$           16.5              -2.63  
$\chi_{b0}(2P)$ $\Upsilon(2S)$           13.3               -5.7  
$\chi_{b2}(3P)$ $\Upsilon(2S)$           6.26              0.703  
$\chi_{b1}(3P)$ $\Upsilon(2S)$           4.52             -0.239  
$\chi_{b0}(3P)$ $\Upsilon(2S)$           1.69             -0.203

### $\Upsilon(3S)$

In [307]:
report, sr = getSumrule("yps_3S", "scr", ("1P1D2D.b-scr", "nSnP.b-scr"), "ELW")
display(sr)
display(report)

,$\sigma$
$\sigma_{PDG}$,0.543
$\sigma_{Deng}$,8.79
$\sigma_{\Gamma}$,-2.54


PDG  $\sigma_{PDG}$   Deng  \
In              Out                                             
$\Upsilon(3S)$  $\chi_{b2}(2P)$   2.66            14.7   3.16   
                $\chi_{b1}(2P)$   2.56              15   2.61   
                $\chi_{b0}(2P)$    1.2              -0   1.21   
                $\chi_{b2}(1P)$  0.201         0.00854   0.14   
                $\chi_{b1}(1P)$ 0.0183         0.00114 0.0005   
                $\chi_{b0}(1P)$ 0.0549              -0  0.097   
$\chi_{b2}(3P)$ $\Upsilon(3S)$     nan            37.1   10.8   
$\chi_{b1}(3P)$ $\Upsilon(3S)$     nan           -19.7   10.3   
$\chi_{b0}(3P)$ $\Upsilon(3S)$     nan           -46.4   7.95   

                                 $\sigma_{Deng}$  ELW_$\Gamma$  \
In              Out                                              
$\Upsilon(3S)$  $\chi_{b2}(2P)$             17.4          2.46   
                $\chi_{b1}(2P)$             15.3          2.22   
                $\chi_{b0}(2P)$               -0          1.31   
                $\chi_{b2}(1P)$          0.00594         0.394   
                $\chi_{b1}(1P)$         3.13e-05        0.0357   
                $\chi_{b0}(1P)$               -0        0.0394   
$\chi_{b2}(3P)$ $\Upsilon(3S)$              28.6            14   
$\chi_{b1}(3P)$ $\Upsilon(3S)$             -15.7            13   
$\chi_{b0}(3P)$ $\Upsilon(3S)$             -36.8            10   

                                 $\sigma_{\Gamma}$  
In              Out                                 
$\Upsilon(3S)$  $\chi_{b2}(2P)$               13.5  
                $\chi_{b1}(2P)$                 13  
                $\chi_{b0}(2P)$                 -0  
                $\chi_{b2}(1P)$             0.0167  
                $\chi_{b1}(1P)$            0.00223  
                $\chi_{b0}(1P)$                 -0  
$\chi_{b2}(3P)$ $\Upsilon(3S)$                37.1  
$\chi_{b1}(3P)$ $\Upsilon(3S)$               -19.7  
$\chi_{b0}(3P)$ $\Upsilon(3S)$               -46.4

## Charmonium sumrules

### $\psi(1S)$

In [308]:
report, sr = getSumrule("psi_1S", "scr", ["c-scr"])
display(sr)
display(report)

,$\sigma$
$\sigma_{PDG}$,-1.1
$\sigma_{Deng}$,-24.4
$\sigma_{\Gamma}$,-33


,,PDG,$\sigma_{PDG}$,Deng,$\sigma_{Deng}$,$\Gamma$,$\sigma_{\Gamma}$
In,Out,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,-0,2.44,-0,2.78,-0
$\eta_{c}(2S)$,$\psi(1S)$,nan,-0.266,2.29,-0.209,2.91,-0.266
$\chi_{c2}(1P)$,$\psi(1S)$,371,57.7,292,45.4,311,48.4
$\chi_{c1}(1P)$,$\psi(1S)$,285,-40.1,319,-44.9,376,-53
$\chi_{c0}(1P)$,$\psi(1S)$,133,-18.4,179,-24.7,204,-28.2


In [309]:
report, sr = getSumrule("psi_1S", "lin", ["c-lin"])
display(sr)
display(report)

,$\sigma$
$\sigma_{PDG}$,-2.19
$\sigma_{Deng}$,-23.8
$\sigma_{\Gamma}$,-30.4


,,PDG,$\sigma_{PDG}$,Deng,$\sigma_{Deng}$,$\Gamma$,$\sigma_{\Gamma}$
In,Out,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,-0,2.39,-0,2.57,-0
$\eta_{c}(2S)$,$\psi(1S)$,nan,-0.277,2.64,-0.241,3.03,-0.277
$\chi_{c2}(1P)$,$\psi(1S)$,371,57,284,43.7,303,46.6
$\chi_{c1}(1P)$,$\psi(1S)$,285,-40.5,306,-43.5,350,-49.8
$\chi_{c0}(1P)$,$\psi(1S)$,133,-18.4,172,-23.7,195,-26.9


### $\psi(2S)$

In [310]:
report, sr = getSumrule("psi_2S", "scr", ["c-scr"])
display(sr)
display(report)

,$\sigma$
$\sigma_{PDG}$,25.8
$\sigma_{Deng}$,42.9
$\sigma_{\Gamma}$,37.4


PDG  $\sigma_{PDG}$  Deng  $\sigma_{Deng}$  \
In         Out                                                            
$\psi(2S)$ $\eta_{c}(2S)$  0.207              -0  0.19               -0   
           $\eta_{c}(1S)$   1.01              -0   7.8               -0   
           $\chi_{c2}(1P)$    27            15.6    46             26.5   
           $\chi_{c1}(1P)$  28.3            10.3    45             16.4   
           $\chi_{c0}(1P)$  29.6              -0    22               -0   

                            $\Gamma$  $\sigma_{\Gamma}$  
In         Out                                           
$\psi(2S)$ $\eta_{c}(2S)$      0.148                 -0  
           $\eta_{c}(1S)$       10.3                 -0  
           $\chi_{c2}(1P)$      42.8               24.7  
           $\chi_{c1}(1P)$      34.9               12.7  
           $\chi_{c0}(1P)$      20.2                 -0

In [311]:
report, sr = getSumrule("psi_2S", "lin", ["c-lin"])
display(sr)
display(report)

,$\sigma$
$\sigma_{PDG}$,25.8
$\sigma_{Deng}$,37.2
$\sigma_{\Gamma}$,36.6


PDG  $\sigma_{PDG}$  Deng  $\sigma_{Deng}$  \
In         Out                                                            
$\psi(2S)$ $\eta_{c}(2S)$  0.207              -0  0.19               -0   
           $\eta_{c}(1S)$   1.01              -0  8.08               -0   
           $\chi_{c2}(1P)$    27            15.5    38             21.9   
           $\chi_{c1}(1P)$  28.3            10.3    42             15.3   
           $\chi_{c0}(1P)$  29.6              -0    22               -0   

                            $\Gamma$  $\sigma_{\Gamma}$  
In         Out                                           
$\psi(2S)$ $\eta_{c}(2S)$      0.158                 -0  
           $\eta_{c}(1S)$       9.63                 -0  
           $\chi_{c2}(1P)$      40.9               23.6  
           $\chi_{c1}(1P)$      35.9               13.1  
           $\chi_{c0}(1P)$        21                 -0

### $\psi(3S)$

In [312]:
report, sr = getSumrule("psi_3S", "scr", ["c-scr"])
display(sr)
display(report)

,$\sigma$
$\sigma_{PDG}$,236
$\sigma_{Deng}$,405
$\sigma_{\Gamma}$,236


PDG  $\sigma_{PDG}$  Deng  $\sigma_{Deng}$  \
In         Out                                                           
$\psi(3S)$ $\eta_{c}(3S)$   nan              -0 0.088               -0   
           $\eta_{c}(2S)$   nan              -0  1.78               -0   
           $\eta_{c}(1S)$   nan              -0  6.76               -0   
           $\chi_{c2}(2P)$  nan             119    82              212   
           $\chi_{c1}(2P)$  nan             114    67              193   
           $\chi_{c0}(2P)$  nan              -0    27               -0   
           $\chi_{c2}(1P)$  nan            1.35   2.5           0.0952   
           $\chi_{c1}(1P)$  nan            1.43   6.7            0.294   
           $\chi_{c0}(1P)$  nan              -0   6.7               -0   

                            $\Gamma$  $\sigma_{\Gamma}$  
In         Out                                           
$\psi(3S)$ $\eta_{c}(3S)$     0.0344                 -0  
           $\eta_{c}(2S)$       2.01                 -0  
           $\eta_{c}(1S)$       9.62                 -0  
           $\chi_{c2}(2P)$      46.2                119  
           $\chi_{c1}(2P)$      39.4                114  
           $\chi_{c0}(2P)$      20.9                 -0  
           $\chi_{c2}(1P)$      35.5               1.35  
           $\chi_{c1}(1P)$      32.6               1.43  
           $\chi_{c0}(1P)$      17.6                 -0

In [313]:
report, sr = getSumrule("psi_3S", "lin", ["c-lin"])
display(sr)
display(report)

,$\sigma$
$\sigma_{PDG}$,460
$\sigma_{Deng}$,463
$\sigma_{\Gamma}$,460


PDG  $\sigma_{PDG}$  Deng  $\sigma_{Deng}$  \
In         Out                                                           
$\psi(3S)$ $\eta_{c}(3S)$   nan               0 0.051                0   
           $\eta_{c}(2S)$   nan              -0  1.65               -0   
           $\eta_{c}(1S)$   nan              -0  6.66               -0   
           $\chi_{c2}(2P)$  nan             168    67              174   
           $\chi_{c1}(2P)$  nan             288    55              289   
           $\chi_{c0}(2P)$  nan              -0    19               -0   
           $\chi_{c2}(1P)$  nan            1.57  0.25          0.00939   
           $\chi_{c1}(1P)$  nan            1.87     4            0.177   
           $\chi_{c0}(1P)$  nan              -0   5.9               -0   

                            $\Gamma$  $\sigma_{\Gamma}$  
In         Out                                           
$\psi(3S)$ $\eta_{c}(3S)$     0.0527                  0  
           $\eta_{c}(2S)$        2.8                 -0  
           $\eta_{c}(1S)$       10.4                 -0  
           $\chi_{c2}(2P)$      64.9                168  
           $\chi_{c1}(2P)$      54.8                288  
           $\chi_{c0}(2P)$      26.8                 -0  
           $\chi_{c2}(1P)$      41.7               1.57  
           $\chi_{c1}(1P)$      42.2               1.87  
           $\chi_{c0}(1P)$      22.9                 -0

In [314]:
raise KeyboardInterrupt("Enough for output")

KeyboardInterrupt: Enough for output

# Sandbox

In [ ]:
def readTechs(particle, cfgnames, prefix):
    stack = []
    for cfgname in cfgnames:
        stack.append(pd.read_csv("output/{}.{}/melsq".format(prefix, cfgname), header=0, index_col=[0,1], names=["In", "Out", "Tot", "E1", "ELW"]))
    res = pd.concat(stack)
    res = res.query("In == '{0}' | Out == '{0}'".format(particle))
    res.index = pd.Index([tuple("${}$".format(code2name(n)) for n in p) for p in res.index.values], names=["In", "Out"])
    return res

In [ ]:
readTechs("chi_b2_1P", ("1P1D2D.b-scr", "nSnP.b-scr"), "scr")

In [ ]:
doReport("scr.E1.1P1D2D.b-scr", "../data/bottomonia.csv", "../data/b-scr.deng2017-1P1D2D.csv", "../data/bottom-trans-PDG.csv")

In [ ]:
doReport("scr.E1.nSnP.b-scr", "../data/bottomonia.csv", "../data/b-scr.deng2017-nSnP.csv", "../data/bottom-trans-PDG.csv")

In [ ]:
doReport("scr.c-scr","../data/charmonia.csv", "../data/c-scr.deng2017.csv", "../data/charm-trans-PDG.csv")

In [ ]:
doReport("lin.c-lin","../data/charmonia.csv", "../data/c-lin.deng2017.csv", "../data/charm-trans-PDG.csv")